In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval
from pandas import Series,DataFrame
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data=pd.read_csv("movies_metadata.csv")

C:\Users\Haikrishnan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data1=data.copy()

In [4]:
data2=data.copy()

In [5]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [6]:
data['original_language'].unique()

array(['en', 'fr', 'zh', 'it', 'fa', 'nl', 'de', 'cn', 'ar', 'es', 'ru',
       'sv', 'ja', 'ko', 'sr', 'bn', 'he', 'pt', 'wo', 'ro', 'hu', 'cy',
       'vi', 'cs', 'da', 'no', 'nb', 'pl', 'el', 'sh', 'xx', 'mk', 'bo',
       'ca', 'fi', 'th', 'sk', 'bs', 'hi', 'tr', 'is', 'ps', 'ab', 'eo',
       'ka', 'mn', 'bm', 'zu', 'uk', 'af', 'la', 'et', 'ku', 'fy', 'lv',
       'ta', 'sl', 'tl', 'ur', 'rw', 'id', 'bg', 'mr', 'lt', 'kk', 'ms',
       'sq', nan, '104.0', 'qu', 'te', 'am', 'jv', 'tg', 'ml', 'hr', 'lo',
       'ay', 'kn', 'eu', 'ne', 'pa', 'ky', 'gl', '68.0', 'uz', 'sm', 'mt',
       '82.0', 'hy', 'iu', 'lb', 'si'], dtype=object)

In [7]:
dr=['adult','budget','homepage','original_title','release_date','revenue','runtime','status','video']

In [8]:
data.drop(dr,axis=1,inplace=True)

In [9]:
data

,belongs_to_collection,genres,id,imdb_id,original_language,overview,popularity,poster_path,production_companies,production_countries,spoken_languages,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,5415.0
1,NaN,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,NaN,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...","[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,NaN,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,tt6209470,fa,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]","[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Rising and falling between a man and woman,Subdue,4.0,1.0
45462,NaN,"[{'id': 18, 'name': 'Drama'}]",111109,tt2028550,tl,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]","[{'iso_639_1': 'tl', 'name': ''}]",NaN,Century of Birthing,9.0,3.0
45463,NaN,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,tt0303758,en,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",A deadly game of wits.,Betrayal,3.8,6.0
45464,NaN,[],227506,tt0008536,en,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",[],NaN,Satan Triumphant,0.0,0.0


In [10]:
data['genres']=data['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [] )

In [11]:
data

,belongs_to_collection,genres,id,imdb_id,original_language,overview,popularity,poster_path,production_companies,production_countries,spoken_languages,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,5415.0
1,NaN,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[Romance, Comedy]",15602,tt0113228,en,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,NaN,"[Comedy, Drama, Romance]",31357,tt0114885,en,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",[Comedy],11862,tt0113041,en,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,NaN,"[Drama, Family]",439050,tt6209470,fa,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]","[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Rising and falling between a man and woman,Subdue,4.0,1.0
45462,NaN,[Drama],111109,tt2028550,tl,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]","[{'iso_639_1': 'tl', 'name': ''}]",NaN,Century of Birthing,9.0,3.0
45463,NaN,"[Action, Drama, Thriller]",67758,tt0303758,en,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",A deadly game of wits.,Betrayal,3.8,6.0
45464,NaN,[],227506,tt0008536,en,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",[],NaN,Satan Triumphant,0.0,0.0


In [12]:
C=data['vote_average'][data['vote_average'].notnull()].mean()
m=data['vote_count'].quantile(0.95)

In [13]:
qual=data[(data['vote_count']>m)&(data['vote_count'].notnull())&(data['vote_average'].notnull())]

In [14]:
qual

,belongs_to_collection,genres,id,imdb_id,original_language,overview,popularity,poster_path,production_companies,production_countries,spoken_languages,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,5415.0
1,NaN,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
5,NaN,"[Action, Crime, Drama, Thriller]",949,tt0113277,en,"Obsessive master thief, Neil McCauley leads a ...",17.9249,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",A Los Angeles Crime Saga,Heat,7.7,1886.0
9,"{'id': 645, 'name': 'James Bond Collection', '...","[Adventure, Action, Thriller]",710,tt0113189,en,James Bond must unmask the mysterious head of ...,14.686,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",No limits. No fears. No substitutes.,GoldenEye,6.6,1194.0
15,NaN,"[Drama, Crime]",524,tt0112641,en,The life of the gambling paradise – Las Vegas ...,10.1374,/xo517ibXBDdYQY81j0WIG7BVcWq.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...","[{'iso_639_1': 'en', 'name': 'English'}]",No one stays at the top forever.,Casino,7.8,1343.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44624,NaN,"[Science Fiction, Thriller]",406990,tt1536537,en,In a world where families are limited to one c...,60.581223,/o6EsOqITcSzcdwD1zxBM9imdxjr.jpg,"[{'name': 'Vendome Pictures', 'id': 7460}, {'n...","[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is...","[{'iso_639_1': 'en', 'name': 'English'}]",Seven sisters. One identity.,What Happened to Monday,7.3,598.0
44632,NaN,"[Action, Thriller]",341013,tt2406566,en,An undercover MI6 agent is sent to Berlin duri...,14.455104,/kV9R5h0Yct1kR8Hf8sJ1nX0Vz4x.jpg,"[{'name': 'Denver and Delilah Productions', 'i...","[{'iso_3166_1': 'SE', 'name': 'Sweden'}, {'iso...","[{'iso_639_1': 'sv', 'name': 'svenska'}, {'iso...",NaN,Atomic Blonde,6.1,748.0
44678,NaN,"[Action, Drama, History, Thriller, War]",374720,tt5013056,en,The miraculous evacuation of Allied soldiers f...,30.938854,/bOXBV303Fgkzn2K4FeKDc0O31q4.jpg,"[{'name': 'Canal+', 'id': 5358}, {'name': 'Stu...","[{'iso_3166_1': 'NL', 'name': 'Netherlands'}, ...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",The event that shaped our world,Dunkirk,7.5,2712.0
44842,"{'id': 8650, 'name': 'Transformers Collection'...","[Action, Science Fiction, Thriller, Adventure]",335988,tt3371366,en,"Autobots and Decepticons are at war, with huma...",39.186819,/s5HQf2Gb3lIO2cRcFwNL9sn1o1o.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]","For one world to live, the other must die.",Transformers: The Last Knight,6.2,1440.0


In [15]:
qual['vote_average']=qual['vote_average'].astype(int)
qual['vote_count']=qual['vote_count'].astype(int)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
qual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2268 entries, 0 to 45014
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   belongs_to_collection  769 non-null    object
 1   genres                 2268 non-null   object
 2   id                     2268 non-null   object
 3   imdb_id                2268 non-null   object
 4   original_language      2268 non-null   object
 5   overview               2268 non-null   object
 6   popularity             2268 non-null   object
 7   poster_path            2268 non-null   object
 8   production_companies   2268 non-null   object
 9   production_countries   2268 non-null   object
 10  spoken_languages       2268 non-null   object
 11  tagline                2165 non-null   object
 12  title                  2268 non-null   object
 13  vote_average           2268 non-null   int32 
 14  vote_count             2268 non-null   int32 
dtypes: int32(2), object(

In [17]:
qual

,belongs_to_collection,genres,id,imdb_id,original_language,overview,popularity,poster_path,production_companies,production_countries,spoken_languages,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7,5415
1,NaN,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6,2413
5,NaN,"[Action, Crime, Drama, Thriller]",949,tt0113277,en,"Obsessive master thief, Neil McCauley leads a ...",17.9249,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",A Los Angeles Crime Saga,Heat,7,1886
9,"{'id': 645, 'name': 'James Bond Collection', '...","[Adventure, Action, Thriller]",710,tt0113189,en,James Bond must unmask the mysterious head of ...,14.686,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",No limits. No fears. No substitutes.,GoldenEye,6,1194
15,NaN,"[Drama, Crime]",524,tt0112641,en,The life of the gambling paradise – Las Vegas ...,10.1374,/xo517ibXBDdYQY81j0WIG7BVcWq.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...","[{'iso_639_1': 'en', 'name': 'English'}]",No one stays at the top forever.,Casino,7,1343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44624,NaN,"[Science Fiction, Thriller]",406990,tt1536537,en,In a world where families are limited to one c...,60.581223,/o6EsOqITcSzcdwD1zxBM9imdxjr.jpg,"[{'name': 'Vendome Pictures', 'id': 7460}, {'n...","[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is...","[{'iso_639_1': 'en', 'name': 'English'}]",Seven sisters. One identity.,What Happened to Monday,7,598
44632,NaN,"[Action, Thriller]",341013,tt2406566,en,An undercover MI6 agent is sent to Berlin duri...,14.455104,/kV9R5h0Yct1kR8Hf8sJ1nX0Vz4x.jpg,"[{'name': 'Denver and Delilah Productions', 'i...","[{'iso_3166_1': 'SE', 'name': 'Sweden'}, {'iso...","[{'iso_639_1': 'sv', 'name': 'svenska'}, {'iso...",NaN,Atomic Blonde,6,748
44678,NaN,"[Action, Drama, History, Thriller, War]",374720,tt5013056,en,The miraculous evacuation of Allied soldiers f...,30.938854,/bOXBV303Fgkzn2K4FeKDc0O31q4.jpg,"[{'name': 'Canal+', 'id': 5358}, {'name': 'Stu...","[{'iso_3166_1': 'NL', 'name': 'Netherlands'}, ...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",The event that shaped our world,Dunkirk,7,2712
44842,"{'id': 8650, 'name': 'Transformers Collection'...","[Action, Science Fiction, Thriller, Adventure]",335988,tt3371366,en,"Autobots and Decepticons are at war, with huma...",39.186819,/s5HQf2Gb3lIO2cRcFwNL9sn1o1o.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]","For one world to live, the other must die.",Transformers: The Last Knight,6,1440


In [18]:
def Weightedrating(x):
    R=x['vote_average']
    v=x['vote_count']
    wrd=((v*R)+(m*C))/(v+m)
    return wrd

In [19]:
qual['wrd']=Weightedrating(qual)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
qual

,belongs_to_collection,genres,id,imdb_id,original_language,overview,popularity,poster_path,production_companies,production_countries,spoken_languages,tagline,title,vote_average,vote_count,wrd
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[Animation, Comedy, Family]",862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7,5415,6.897470
1,NaN,"[Adventure, Fantasy, Family]",8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6,2413,5.941799
5,NaN,"[Action, Crime, Drama, Thriller]",949,tt0113277,en,"Obsessive master thief, Neil McCauley leads a ...",17.9249,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",A Los Angeles Crime Saga,Heat,7,1886,6.741509
9,"{'id': 645, 'name': 'James Bond Collection', '...","[Adventure, Action, Thriller]",710,tt0113189,en,James Bond must unmask the mysterious head of ...,14.686,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",No limits. No fears. No substitutes.,GoldenEye,6,1194,5.898220
15,NaN,"[Drama, Crime]",524,tt0112641,en,The life of the gambling paradise – Las Vegas ...,10.1374,/xo517ibXBDdYQY81j0WIG7BVcWq.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...","[{'iso_639_1': 'en', 'name': 'English'}]",No one stays at the top forever.,Casino,7,1343,6.662522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44624,NaN,"[Science Fiction, Thriller]",406990,tt1536537,en,In a world where families are limited to one c...,60.581223,/o6EsOqITcSzcdwD1zxBM9imdxjr.jpg,"[{'name': 'Vendome Pictures', 'id': 7460}, {'n...","[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is...","[{'iso_639_1': 'en', 'name': 'English'}]",Seven sisters. One identity.,What Happened to Monday,7,598,6.418897
44632,NaN,"[Action, Thriller]",341013,tt2406566,en,An undercover MI6 agent is sent to Berlin duri...,14.455104,/kV9R5h0Yct1kR8Hf8sJ1nX0Vz4x.jpg,"[{'name': 'Denver and Delilah Productions', 'i...","[{'iso_3166_1': 'SE', 'name': 'Sweden'}, {'iso...","[{'iso_639_1': 'sv', 'name': 'svenska'}, {'iso...",NaN,Atomic Blonde,6,748,5.859816
44678,NaN,"[Action, Drama, History, Thriller, War]",374720,tt5013056,en,The miraculous evacuation of Allied soldiers f...,30.938854,/bOXBV303Fgkzn2K4FeKDc0O31q4.jpg,"[{'name': 'Canal+', 'id': 5358}, {'name': 'Stu...","[{'iso_3166_1': 'NL', 'name': 'Netherlands'}, ...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",The event that shaped our world,Dunkirk,7,2712,6.809378
44842,"{'id': 8650, 'name': 'Transformers Collection'...","[Action, Science Fiction, Thriller, Adventure]",335988,tt3371366,en,"Autobots and Decepticons are at war, with huma...",39.186819,/s5HQf2Gb3lIO2cRcFwNL9sn1o1o.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]","For one world to live, the other must die.",Transformers: The Last Knight,6,1440,5.911581


In [21]:
qual['production_companies']=qual['production_companies'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
data['production_companies']=data['production_companies'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])

In [23]:
qual['belongs_to_collection']=qual['belongs_to_collection'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
qual.drop('belongs_to_collection',axis=1,inplace=True)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [25]:
qual.sort_values('wrd',ascending=False).head(20)

,genres,id,imdb_id,original_language,overview,popularity,poster_path,production_companies,production_countries,spoken_languages,tagline,title,vote_average,vote_count,wrd
15480,"[Action, Thriller, Science Fiction, Mystery, A...",27205,tt1375666,en,"Cobb, a skilled thief who commits corporate es...",29.1081,/qmDpIHrmpJINaRKAfWQfftjCdyi.jpg,"[Legendary Pictures, Warner Bros., Syncopy]","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'iso_639_1': 'en', 'name': 'English'}]",Your mind is the scene of the crime.,Inception,8,14075,7.928755
12481,"[Drama, Action, Crime, Thriller]",155,tt0468569,en,Batman raises the stakes in his war on crime. ...,123.167,/1hRoyzDtpgMU7Dz4JF22RANzQO7.jpg,"[DC Comics, Legendary Pictures, Warner Bros., ...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Why So Serious?,The Dark Knight,8,12269,7.918626
22879,"[Adventure, Drama, Science Fiction]",157336,tt0816692,en,Interstellar chronicles the adventures of a gr...,32.2135,/nBNZadXqJSdt05SHLqgT0HuC5Gm.jpg,"[Paramount Pictures, Legendary Pictures, Warne...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...","[{'iso_639_1': 'en', 'name': 'English'}]",Mankind was born on Earth. It was never meant ...,Interstellar,8,11187,7.911049
2843,[Drama],550,tt0137523,en,A ticking-time-bomb insomniac and a slippery s...,63.8696,/adw6Lq9FiC9zjYEpOqfq03ituwp.jpg,"[Twentieth Century Fox Film Corporation, Regen...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...","[{'iso_639_1': 'en', 'name': 'English'}]",Mischief. Mayhem. Soap.,Fight Club,8,9678,7.897775
4863,"[Adventure, Fantasy, Action]",120,tt0120737,en,"Young hobbit Frodo Baggins, after inheriting a...",32.0707,/bxVxZb5O9OxCO0oRUNdCnpy9NST.jpg,"[WingNut Films, New Line Cinema, The Saul Zaen...","[{'iso_3166_1': 'NZ', 'name': 'New Zealand'}, ...","[{'iso_639_1': 'en', 'name': 'English'}]",One ring to rule them all,The Lord of the Rings: The Fellowship of the Ring,8,8892,7.889160
292,"[Thriller, Crime]",680,tt0110912,en,"A burger-loving hit man, his philosophical par...",140.95,/dM2w364MScsjFf8pfMbaWUcWrR.jpg,"[Miramax Films, A Band Apart, Jersey Films]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Just because you are a character doesn't mean ...,Pulp Fiction,8,8670,7.886457
314,"[Drama, Crime]",278,tt0111161,en,Framed in the 1940s for the double murder of h...,51.6454,/9O7gLzmreU0nGkIB6K3BsJbzvNv.jpg,"[Castle Rock Entertainment, Warner Bros.]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,8,8358,7.882427
7000,"[Adventure, Fantasy, Action]",122,tt0167260,en,Aragorn is revealed as the heir to the ancient...,29.3244,/uexxR7Kw1qYbZk0RYaF9Rx5ykbj.jpg,"[WingNut Films, New Line Cinema]","[{'iso_3166_1': 'NZ', 'name': 'New Zealand'}, ...","[{'iso_639_1': 'en', 'name': 'English'}]",The eye of the enemy is moving.,The Lord of the Rings: The Return of the King,8,8226,7.880635
351,"[Comedy, Drama, Romance]",13,tt0109830,en,A man with a low IQ has accomplished great thi...,48.3072,/yE5d3BUhE8hCnkMUJOo1QDoOGNz.jpg,[Paramount Pictures],"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]","The world will never be the same, once you've ...",Forrest Gump,8,8147,7.879536
5814,"[Adventure, Fantasy, Action]",121,tt0167261,en,Frodo and Sam are trekking to Mordor to destro...,29.4235,/wf3v0Pn09jnT5HSaYal7Ami3bdA.jpg,"[WingNut Films, New Line Cinema, The Saul Zaen...","[{'iso_3166_1': 'NZ', 'name': 'New Zealand'}, ...","[{'iso_639_1': 'en', 'name': 'English'}]",A New Power Is Rising.,The Lord of the Rings: The Two Towers,8,7641,7.871988


In [26]:
s=qual.apply(lambda x:pd.Series(x['genres']),axis=1).stack().reset_index(level=1,drop=True)

s.name='genre'

In [27]:
s

0              Animation
0                 Comedy
0                 Family
1              Adventure
1                Fantasy
              ...       
45014             Action
45014            Western
45014    Science Fiction
45014            Fantasy
45014             Horror
Name: genre, Length: 6244, dtype: object

In [28]:
a=qual.join(s)

In [29]:
a.drop('genres',axis=1,inplace=True)

In [30]:
a

,id,imdb_id,original_language,overview,popularity,poster_path,production_companies,production_countries,spoken_languages,tagline,title,vote_average,vote_count,wrd,genre
0,862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7,5415,6.897470,Animation
0,862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7,5415,6.897470,Comedy
0,862,tt0114709,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7,5415,6.897470,Family
1,8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6,2413,5.941799,Adventure
1,8844,tt0113497,en,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6,2413,5.941799,Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45014,353491,tt1648190,en,"The last Gunslinger, Roland Deschain, has been...",50.903593,/i9GUSgddIqrroubiLsvvMRYyRy0.jpg,"[Imagine Entertainment, Weed Road Pictures, Me...","[{'iso_3166_1': 'ZA', 'name': 'South Africa'},...","[{'iso_639_1': 'en', 'name': 'English'}]",There are other worlds than these,The Dark Tower,5,688,5.239128,Action
45014,353491,tt1648190,en,"The last Gunslinger, Roland Deschain, has been...",50.903593,/i9GUSgddIqrroubiLsvvMRYyRy0.jpg,"[Imagine Entertainment, Weed Road Pictures, Me...","[{'iso_3166_1': 'ZA', 'name': 'South Africa'},...","[{'iso_639_1': 'en', 'name': 'English'}]",There are other worlds than these,The Dark Tower,5,688,5.239128,Western
45014,353491,tt1648190,en,"The last Gunslinger, Roland Deschain, has been...",50.903593,/i9GUSgddIqrroubiLsvvMRYyRy0.jpg,"[Imagine Entertainment, Weed Road Pictures, Me...","[{'iso_3166_1': 'ZA', 'name': 'South Africa'},...","[{'iso_639_1': 'en', 'name': 'English'}]",There are other worlds than these,The Dark Tower,5,688,5.239128,Science Fiction
45014,353491,tt1648190,en,"The last Gunslinger, Roland Deschain, has been...",50.903593,/i9GUSgddIqrroubiLsvvMRYyRy0.jpg,"[Imagine Entertainment, Weed Road Pictures, Me...","[{'iso_3166_1': 'ZA', 'name': 'South Africa'},...","[{'iso_639_1': 'en', 'name': 'English'}]",There are other worlds than these,The Dark Tower,5,688,5.239128,Fantasy


In [31]:
a[a['genre']=='Romance']

,id,imdb_id,original_language,overview,popularity,poster_path,production_companies,production_countries,spoken_languages,tagline,title,vote_average,vote_count,wrd,genre
38,9603,tt0112697,en,"Shallow, rich and socially successful Cher is ...",9.88238,/i8gEHh2sszB6YWLC0jl559sxAeN.jpg,[Paramount Pictures],"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'es', 'name': 'Español'}, {'iso...",Sex. Clothes. Popularity. Is there a problem h...,Clueless,6,828,5.868702,Romance
212,76,tt0112471,en,"A dialogue marathon of a film, this fairytale ...",11.305,/jsQy4ZbPHA8hE2O6QU05PpofI61.jpg,"[Detour Film Production, Warner Bros.]","[{'iso_3166_1': 'AT', 'name': 'Austria'}, {'is...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Can the greatest romance of your life last onl...,Before Sunrise,7,984,6.577082,Romance
249,628,tt0110148,en,"A vampire relates his epic life story of love,...",11.9702,/hldXwwViSfHJS0kIJr07KBGmHJI.jpg,[Geffen Pictures],"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Drink from me and live forever,Interview with the Vampire,7,1558,6.698947,Romance
262,4476,tt0110322,en,An epic tale of three brothers and their fathe...,12.199,/uh0sJcx3SLtclJSuKAXl6Tt6AV0.jpg,"[Bedford Falls Productions, TriStar Pictures, ...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'kw', 'name': ''}, {'iso_639_1'...",After the Fall from Innocence the Legend begins.,Legends of the Fall,7,636,6.439535,Romance
332,1587,tt0108550,en,Gilbert has to care for his brother Arnie and ...,10.3919,/jQLoOPwNxyILwDbdhn9DsfQz0Sg.jpg,[Paramount Pictures],"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",A film about the love you find...In the last p...,What's Eating Gilbert Grape,7,943,6.564489,Romance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41488,365942,tt3922818,en,A young man raised by scientists on Mars retur...,11.824404,/AjZ22T1tFWlmwi4jIsV0fsvnI6c.jpg,"[Southpaw Entertainment, STX Entertainment, Lo...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",What's your favorite thing about Earth?,The Space Between Us,7,564,6.399100,Romance
41536,274870,tt1355644,en,A spacecraft traveling to a distant colony pla...,20.303632,/5gJkVIVU7FDp7AfRAbPSvvdbre2.jpg,"[Columbia Pictures, Village Roadshow Pictures,...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",There is a reason they woke up.,Passengers,6,4134,5.963726,Romance
42222,321612,tt2771200,en,A live-action adaptation of Disney's version o...,287.253654,/tWqifoYuwLETmmasnGHO7xBjEtt.jpg,"[Walt Disney Pictures, Mandeville Films]","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'iso_639_1': 'en', 'name': 'English'}]",Be our guest.,Beauty and the Beast,6,5530,5.972217,Romance
42355,341174,tt4465564,en,When a wounded Christian Grey tries to entice ...,29.130443,/wnVHDbGz7RvDAHFAsVVT88FxhrP.jpg,"[Universal Pictures, Perfect World Pictures (B...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Every fairy tale has a dark side.,Fifty Shades Darker,6,2341,5.940289,Romance


In [32]:
def get_rec(genre,percentile=0.85):
    b=a[a['genre']==genre]
    rec= b[['title','tagline','genre','vote_average','wrd']].sort_values('wrd',ascending=False).head(250)
    
    return rec

In [33]:
get_rec('Action').head(10)

,title,tagline,genre,vote_average,wrd
15480,Inception,Your mind is the scene of the crime.,Action,8,7.928755
12481,The Dark Knight,Why So Serious?,Action,8,7.918626
4863,The Lord of the Rings: The Fellowship of the Ring,One ring to rule them all,Action,8,7.889160
7000,The Lord of the Rings: The Return of the King,The eye of the enemy is moving.,Action,8,7.880635
5814,The Lord of the Rings: The Two Towers,A New Power Is Rising.,Action,8,7.871988
256,Star Wars,"A long time ago in a galaxy far, far away...",Action,8,7.856670
1154,The Empire Strikes Back,The Adventure Continues...,Action,8,7.839288
4135,Scarface,The world is yours...,Action,8,7.700464
9430,Oldboy,"15 years of imprisonment, five days of vengeance",Action,8,7.575309
1910,Seven Samurai,The Mighty Warriors Who Became the Seven Natio...,Action,8,7.220439


In [34]:
get_rec('Romance').head(10)

,title,tagline,genre,vote_average,wrd
351,Forrest Gump,"The world will never be the same, once you've ...",Romance,8,7.879536
10309,Dilwale Dulhania Le Jayenge,Come... Fall In Love,Romance,9,7.659636
876,Vertigo,Alfred Hitchcock engulfs you in a whirlpool of...,Romance,8,7.352320
40251,Your Name.,NaN,Romance,8,7.293922
883,Some Like It Hot,The movie too HOT for words!,Romance,8,7.185423
1132,Cinema Paradiso,"A celebration of youth, friendship, and the ev...",Romance,8,7.184781
19901,Paperman,"Delicate, charming and sweet.",Romance,8,7.114985
37863,Sing Street,Boy meets girl. Girl unimpressed. Boy starts b...,Romance,8,7.062830
1639,Titanic,Nothing on Earth could come between them.,Romance,7,6.926902
882,The Apartment,"Movie-wise, there has never been anything like...",Romance,8,6.890882


In [35]:
data.loc[data['title']=='Dilwale Dulhania Le Jayenge']

,belongs_to_collection,genres,id,imdb_id,original_language,overview,popularity,poster_path,production_companies,production_countries,spoken_languages,tagline,title,vote_average,vote_count
10309,NaN,"[Comedy, Drama, Romance]",19404,tt0112870,hi,"Raj is a rich, carefree, happy-go-lucky second...",34.457,/2gvbZMtV1Zsl7FedJa5ysbpBx2G.jpg,[Yash Raj Films],"[{'iso_3166_1': 'IN', 'name': 'India'}]","[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Come... Fall In Love,Dilwale Dulhania Le Jayenge,9.1,661.0


In [36]:
small_link=pd.read_csv("datasets_3405_6663_links_small.csv")

In [37]:
small_link['tmdbId']=small_link[small_link['tmdbId'].notnull()]['tmdbId'].astype(int)

In [38]:
small_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [39]:
small_tmdb=small_link[small_link['tmdbId'].notnull()]['tmdbId']

    

In [40]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [41]:
small_tmdb

0          862.0
1         8844.0
2        15602.0
3        31357.0
4        11862.0
          ...   
9120    402672.0
9121    315011.0
9122    391698.0
9123    137608.0
9124    410803.0
Name: tmdbId, Length: 9112, dtype: float64

In [42]:
data1.drop(data1[data1['id']=='2014-01-01'].index,inplace=True)

In [43]:
data1.drop(data1[data1['id']=='1997-08-20'].index,inplace=True)

In [44]:
data1.drop(data1[data1['id']=='2012-09-29'].index,inplace=True)

In [45]:
data1['id']=data1['id'].astype(float)

In [46]:
data1

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357.0,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862.0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050.0,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109.0,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758.0,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506.0,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [47]:
small_data=data1[data1['id'].isin(small_tmdb)]

In [48]:
small_data.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [49]:
dr=['adult','budget','homepage','release_date','revenue','runtime','spoken_languages','status','video']

In [50]:
small_data.drop(dr,axis=1,inplace=True)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [51]:
dr=['poster_path','production_companies','production_countries']
small_data.drop(dr,axis=1,inplace=True)

In [52]:
small_data

,belongs_to_collection,genres,id,imdb_id,original_language,original_title,overview,popularity,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,NaN,Toy Story,7.7,5415.0
1,NaN,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,NaN,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357.0,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...","[{'id': 35, 'name': 'Comedy'}]",11862.0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...
40224,NaN,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",315011.0,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,9.285519,A god incarnate. A city doomed.,Shin Godzilla,6.6,152.0
40503,NaN,"[{'id': 99, 'name': 'Documentary'}, {'id': 104...",391698.0,tt2531318,en,The Beatles: Eight Days a Week - The Touring Y...,"The band stormed Europe in 1963, and, in 1964,...",7.078301,The band you know. The story you don't.,The Beatles: Eight Days a Week - The Touring Y...,7.6,92.0
44821,"{'id': 34055, 'name': 'Pokémon Collection', 'p...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",10991.0,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,6.480376,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,144.0
44826,"{'id': 34055, 'name': 'Pokémon Collection', 'p...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",12600.0,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",6.080108,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0


In [53]:
small_data['genres']=small_data['genres'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
small_data

,belongs_to_collection,genres,id,imdb_id,original_language,original_title,overview,popularity,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[Animation, Comedy, Family]",862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,NaN,Toy Story,7.7,5415.0
1,NaN,"[Adventure, Fantasy, Family]",8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...","[Romance, Comedy]",15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,NaN,"[Comedy, Drama, Romance]",31357.0,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",[Comedy],11862.0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...
40224,NaN,"[Action, Adventure, Drama, Horror, Science Fic...",315011.0,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,9.285519,A god incarnate. A city doomed.,Shin Godzilla,6.6,152.0
40503,NaN,"[Documentary, Music]",391698.0,tt2531318,en,The Beatles: Eight Days a Week - The Touring Y...,"The band stormed Europe in 1963, and, in 1964,...",7.078301,The band you know. The story you don't.,The Beatles: Eight Days a Week - The Touring Y...,7.6,92.0
44821,"{'id': 34055, 'name': 'Pokémon Collection', 'p...","[Adventure, Fantasy, Animation, Action, Family]",10991.0,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,6.480376,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,144.0
44826,"{'id': 34055, 'name': 'Pokémon Collection', 'p...","[Adventure, Fantasy, Animation, Science Fictio...",12600.0,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",6.080108,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0


In [55]:
small_data.drop('belongs_to_collection',axis=1,inplace=True)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [56]:
small_data.drop('original_title',axis=1,inplace=True)

In [57]:
small_data['tagline']=small_data['tagline'].fillna('')

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
small_data.drop('imdb_id',axis=1,inplace=True)


In [59]:
small_data['overview'].fillna('',inplace=True)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [60]:
small_data.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 9099 entries, 0 to 45265
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genres             9099 non-null   object 
 1   id                 9099 non-null   float64
 2   original_language  9099 non-null   object 
 3   overview           9099 non-null   object 
 4   popularity         9099 non-null   object 
 5   tagline            9099 non-null   object 
 6   title              9099 non-null   object 
 7   vote_average       9099 non-null   float64
 8   vote_count         9099 non-null   float64
dtypes: float64(3), object(6)
memory usage: 710.9+ KB


In [61]:
small_data['overview']


0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
40224    From the mind behind Evangelion comes a hit la...
40503    The band stormed Europe in 1963, and, in 1964,...
44821    When Molly Hale's sadness of her father's disa...
44826    All your favorite Pokémon characters are back,...
45265    While holidaying in the French Alps, a Swedish...
Name: overview, Length: 9099, dtype: object

In [62]:
small_data['description']=small_data['overview']+small_data['tagline']

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
small_data['description'].fillna('',inplace=True)

In [64]:
small_data

,genres,id,original_language,overview,popularity,tagline,title,vote_average,vote_count,description
0,"[Animation, Comedy, Family]",862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,,Toy Story,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ..."
1,"[Adventure, Fantasy, Family]",8844.0,en,When siblings Judy and Peter discover an encha...,17.0155,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,When siblings Judy and Peter discover an encha...
2,"[Romance, Comedy]",15602.0,en,A family wedding reignites the ancient feud be...,11.7129,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,A family wedding reignites the ancient feud be...
3,"[Comedy, Drama, Romance]",31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.85949,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"Cheated on, mistreated and stepped on, the wom..."
4,[Comedy],11862.0,en,Just when George Banks has recovered from his ...,8.38752,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...,...,...,...
40224,"[Action, Adventure, Drama, Horror, Science Fic...",315011.0,ja,From the mind behind Evangelion comes a hit la...,9.285519,A god incarnate. A city doomed.,Shin Godzilla,6.6,152.0,From the mind behind Evangelion comes a hit la...
40503,"[Documentary, Music]",391698.0,en,"The band stormed Europe in 1963, and, in 1964,...",7.078301,The band you know. The story you don't.,The Beatles: Eight Days a Week - The Touring Y...,7.6,92.0,"The band stormed Europe in 1963, and, in 1964,..."
44821,"[Adventure, Fantasy, Animation, Action, Family]",10991.0,ja,When Molly Hale's sadness of her father's disa...,6.480376,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,144.0,When Molly Hale's sadness of her father's disa...
44826,"[Adventure, Fantasy, Animation, Science Fictio...",12600.0,ja,"All your favorite Pokémon characters are back,...",6.080108,,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0,"All your favorite Pokémon characters are back,..."


In [65]:
small_data=small_data.reset_index()

In [66]:
small_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9099 entries, 0 to 9098
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              9099 non-null   int64  
 1   genres             9099 non-null   object 
 2   id                 9099 non-null   float64
 3   original_language  9099 non-null   object 
 4   overview           9099 non-null   object 
 5   popularity         9099 non-null   object 
 6   tagline            9099 non-null   object 
 7   title              9099 non-null   object 
 8   vote_average       9099 non-null   float64
 9   vote_count         9099 non-null   float64
 10  description        9099 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 782.1+ KB


In [67]:
titles=small_data['title']
indices=pd.Series(small_data.index,index=small_data['title'])

In [68]:
tf=TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
tfidf_matrix=tf.fit_transform(small_data['description'])

In [69]:
tfidf_matrix.shape

(9099, 268124)

In [70]:
cosine_mat=linear_kernel(tfidf_matrix,tfidf_matrix)

In [71]:
cosine_mat[9098]

array([0.        , 0.        , 0.00472155, ..., 0.        , 0.        ,
       1.        ])

In [72]:
def get_sugg(title):
    idx=indices[title]
    cos_sim=list(enumerate(cosine_mat[idx]))

    cos_sim=sorted(cos_sim, key=lambda x:x[1],reverse=True)

    cos_sim=cos_sim[1:31]
    movie_indices=[i[0] for i in cos_sim]
    return titles.iloc[movie_indices]

In [73]:
get_sugg('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

In [74]:
credits=pd.read_csv('credits.csv')
keywords=pd.read_csv('keywords.csv')

In [75]:
data2

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [76]:
dr=['adult','budget','homepage','original_title','release_date','revenue','runtime','status','video','poster_path','production_companies','production_countries']

In [77]:
data2.drop(dr,axis=1,inplace=True)

In [78]:
dr=['belongs_to_collection','imdb_id','spoken_languages']


In [79]:
data2.drop(dr,axis=1,inplace=True)

In [80]:
data2['genres']=data2['genres'].apply(literal_eval).apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])

In [81]:
data2

,genres,id,original_language,overview,popularity,tagline,title,vote_average,vote_count
0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,NaN,Toy Story,7.7,5415.0
1,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.0155,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.85949,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.38752,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...
45461,"[Drama, Family]",439050,fa,Rising and falling between a man and woman.,0.072051,Rising and falling between a man and woman,Subdue,4.0,1.0
45462,[Drama],111109,tl,An artist struggles to finish his work while a...,0.178241,NaN,Century of Birthing,9.0,3.0
45463,"[Action, Drama, Thriller]",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,A deadly game of wits.,Betrayal,3.8,6.0
45464,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,NaN,Satan Triumphant,0.0,0.0


In [82]:
credits['crew'].iloc[2]

"[{'credit_id': '52fe466a9251416c75077a89', 'department': 'Directing', 'gender': 2, 'id': 26502, 'job': 'Director', 'name': 'Howard Deutch', 'profile_path': '/68Vae1HkU1NxQZ6KEmuxIpno7c9.jpg'}, {'credit_id': '52fe466b9251416c75077aa3', 'department': 'Writing', 'gender': 2, 'id': 16837, 'job': 'Characters', 'name': 'Mark Steven Johnson', 'profile_path': '/6trChNn3o2bi4i2ipgMEAytwmZp.jpg'}, {'credit_id': '52fe466b9251416c75077aa9', 'department': 'Writing', 'gender': 2, 'id': 16837, 'job': 'Writer', 'name': 'Mark Steven Johnson', 'profile_path': '/6trChNn3o2bi4i2ipgMEAytwmZp.jpg'}, {'credit_id': '5675eb4b92514179dd003933', 'department': 'Crew', 'gender': 2, 'id': 1551320, 'job': 'Sound Recordist', 'name': 'Jack Keller', 'profile_path': None}]"

In [83]:
credits['cast'].iloc[2]

"[{'cast_id': 2, 'character': 'Max Goldman', 'credit_id': '52fe466a9251416c75077a8d', 'gender': 2, 'id': 6837, 'name': 'Walter Matthau', 'order': 0, 'profile_path': '/xJVkvprOnzP5Zdh5y63y8HHniDZ.jpg'}, {'cast_id': 3, 'character': 'John Gustafson', 'credit_id': '52fe466a9251416c75077a91', 'gender': 2, 'id': 3151, 'name': 'Jack Lemmon', 'order': 1, 'profile_path': '/chZmNRYMtqkiDlatprGDH4BzGqG.jpg'}, {'cast_id': 4, 'character': 'Ariel Gustafson', 'credit_id': '52fe466a9251416c75077a95', 'gender': 1, 'id': 13567, 'name': 'Ann-Margret', 'order': 2, 'profile_path': '/jx5lTaJ5VXZHYB52gaOTAZ9STZk.jpg'}, {'cast_id': 5, 'character': 'Maria Sophia Coletta Ragetti', 'credit_id': '52fe466a9251416c75077a99', 'gender': 1, 'id': 16757, 'name': 'Sophia Loren', 'order': 3, 'profile_path': '/emKLhbji1c7BjcA2DdbWf0EP9zH.jpg'}, {'cast_id': 6, 'character': 'Melanie Gustafson', 'credit_id': '52fe466a9251416c75077a9d', 'gender': 1, 'id': 589, 'name': 'Daryl Hannah', 'order': 4, 'profile_path': '/4LLmp6AQdlj6

In [84]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [85]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [86]:
data2.drop(data2[data2['id']=='1997-08-20'].index,inplace=True)

In [87]:
data2.drop(data2[data2['id']=='2012-09-29'].index,inplace=True)

In [88]:
data2.drop(data2[data2['id']=='2014-01-01'].index,inplace=True)

In [89]:
data2['id']=data2['id'].astype(int)

In [90]:
data2=data2.merge(credits,on='id')
data2=data2.merge(keywords,on='id')

In [91]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46628 entries, 0 to 46627
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genres             46628 non-null  object 
 1   id                 46628 non-null  int32  
 2   original_language  46617 non-null  object 
 3   overview           45633 non-null  object 
 4   popularity         46624 non-null  object 
 5   tagline            20783 non-null  object 
 6   title              46624 non-null  object 
 7   vote_average       46624 non-null  float64
 8   vote_count         46624 non-null  float64
 9   cast               46628 non-null  object 
 10  crew               46628 non-null  object 
 11  keywords           46628 non-null  object 
dtypes: float64(2), int32(1), object(9)
memory usage: 4.4+ MB


In [92]:
small_tmdb=small_tmdb.astype(int)

In [93]:
smol_data=data2[data2['id'].isin(small_tmdb)]

In [94]:
smol_data

,genres,id,original_language,overview,popularity,tagline,title,vote_average,vote_count,cast,crew,keywords
0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.0155,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.85949,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.38752,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...,...,...,...,...,...,...,...,...,...,...
40952,[Drama],159550,en,A man must cope with the loss of his wife and ...,0.038998,NaN,The Last Brickmaker in America,7.0,1.0,"[{'cast_id': 1, 'character': 'Henry Cobb', 'cr...","[{'credit_id': '544475aac3a36819fb000578', 'de...","[{'id': 6054, 'name': 'friendship'}, {'id': 20..."
41172,"[Thriller, Romance]",392572,hi,"Rustom Pavri, an honourable officer of the Ind...",7.333139,Decorated Officer. Devoted Family Man. Defendi...,Rustom,7.3,25.0,"[{'cast_id': 0, 'character': 'Rustom Pavri', '...","[{'credit_id': '5951baf692514129c4016600', 'de...","[{'id': 10540, 'name': 'bollywood'}]"
41225,"[Adventure, Drama, History, Romance]",402672,hi,"Village lad Sarman is drawn to big, bad Mohenj...",1.423358,NaN,Mohenjo Daro,6.7,26.0,"[{'cast_id': 0, 'character': 'Sarman', 'credit...","[{'credit_id': '57cd5d3592514179d50018e8', 'de...","[{'id': 10540, 'name': 'bollywood'}]"
41391,"[Action, Adventure, Drama, Horror, Science Fic...",315011,ja,From the mind behind Evangelion comes a hit la...,9.285519,A god incarnate. A city doomed.,Shin Godzilla,6.6,152.0,"[{'cast_id': 4, 'character': 'Rando Yaguchi : ...","[{'credit_id': '560892fa92514177550018b2', 'de...","[{'id': 1299, 'name': 'monster'}, {'id': 7671,..."


In [95]:
def get_director(x):
    for i in x:
        if i['job']=='Director':
            return i['name']
    return np.nan

In [96]:
smol_data['crew']=smol_data['crew'].apply(literal_eval)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [97]:
smol_data['cast']=smol_data['cast'].apply(literal_eval)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
smol_data['keywords']=smol_data['keywords'].apply(literal_eval)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
smol_data['crew_size']=smol_data['crew'].apply(lambda x:len(x))
smol_data['cast_size']=smol_data['cast'].apply(lambda x:len(x))

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [100]:
smol_data['director']=smol_data['crew'].apply(get_director)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [101]:
smol_data

,genres,id,original_language,overview,popularity,tagline,title,vote_average,vote_count,cast,crew,keywords,crew_size,cast_size,director
0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",106,13,John Lasseter
1,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.0155,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",16,26,Joe Johnston
2,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",4,7,Howard Deutch
3,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.85949,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",10,10,Forest Whitaker
4,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.38752,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",7,12,Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40952,[Drama],159550,en,A man must cope with the loss of his wife and ...,0.038998,NaN,The Last Brickmaker in America,7.0,1.0,"[{'cast_id': 1, 'character': 'Henry Cobb', 'cr...","[{'credit_id': '544475aac3a36819fb000578', 'de...","[{'id': 6054, 'name': 'friendship'}, {'id': 20...",2,7,Gregg Champion
41172,"[Thriller, Romance]",392572,hi,"Rustom Pavri, an honourable officer of the Ind...",7.333139,Decorated Officer. Devoted Family Man. Defendi...,Rustom,7.3,25.0,"[{'cast_id': 0, 'character': 'Rustom Pavri', '...","[{'credit_id': '5951baf692514129c4016600', 'de...","[{'id': 10540, 'name': 'bollywood'}]",16,14,Tinu Suresh Desai
41225,"[Adventure, Drama, History, Romance]",402672,hi,"Village lad Sarman is drawn to big, bad Mohenj...",1.423358,NaN,Mohenjo Daro,6.7,26.0,"[{'cast_id': 0, 'character': 'Sarman', 'credit...","[{'credit_id': '57cd5d3592514179d50018e8', 'de...","[{'id': 10540, 'name': 'bollywood'}]",16,12,Ashutosh Gowariker
41391,"[Action, Adventure, Drama, Horror, Science Fic...",315011,ja,From the mind behind Evangelion comes a hit la...,9.285519,A god incarnate. A city doomed.,Shin Godzilla,6.6,152.0,"[{'cast_id': 4, 'character': 'Rando Yaguchi : ...","[{'credit_id': '560892fa92514177550018b2', 'de...","[{'id': 1299, 'name': 'monster'}, {'id': 7671,...",27,49,Hideaki Anno


In [102]:
smol_data['keywords']=smol_data['keywords'].apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])


C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [103]:
smol_data

,genres,id,original_language,overview,popularity,tagline,title,vote_average,vote_count,cast,crew,keywords,crew_size,cast_size,director
0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,NaN,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",106,13,John Lasseter
1,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.0155,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",16,26,Joe Johnston
2,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o...",4,7,Howard Deutch
3,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.85949,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",10,10,Forest Whitaker
4,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.38752,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug...",7,12,Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40952,[Drama],159550,en,A man must cope with the loss of his wife and ...,0.038998,NaN,The Last Brickmaker in America,7.0,1.0,"[{'cast_id': 1, 'character': 'Henry Cobb', 'cr...","[{'credit_id': '544475aac3a36819fb000578', 'de...","[friendship, brick making]",2,7,Gregg Champion
41172,"[Thriller, Romance]",392572,hi,"Rustom Pavri, an honourable officer of the Ind...",7.333139,Decorated Officer. Devoted Family Man. Defendi...,Rustom,7.3,25.0,"[{'cast_id': 0, 'character': 'Rustom Pavri', '...","[{'credit_id': '5951baf692514129c4016600', 'de...",[bollywood],16,14,Tinu Suresh Desai
41225,"[Adventure, Drama, History, Romance]",402672,hi,"Village lad Sarman is drawn to big, bad Mohenj...",1.423358,NaN,Mohenjo Daro,6.7,26.0,"[{'cast_id': 0, 'character': 'Sarman', 'credit...","[{'credit_id': '57cd5d3592514179d50018e8', 'de...",[bollywood],16,12,Ashutosh Gowariker
41391,"[Action, Adventure, Drama, Horror, Science Fic...",315011,ja,From the mind behind Evangelion comes a hit la...,9.285519,A god incarnate. A city doomed.,Shin Godzilla,6.6,152.0,"[{'cast_id': 4, 'character': 'Rando Yaguchi : ...","[{'credit_id': '560892fa92514177550018b2', 'de...","[monster, godzilla, giant monster, destruction...",27,49,Hideaki Anno


In [104]:
smol_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9219 entries, 0 to 41669
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genres             9219 non-null   object 
 1   id                 9219 non-null   int32  
 2   original_language  9219 non-null   object 
 3   overview           9207 non-null   object 
 4   popularity         9219 non-null   object 
 5   tagline            7082 non-null   object 
 6   title              9219 non-null   object 
 7   vote_average       9219 non-null   float64
 8   vote_count         9219 non-null   float64
 9   cast               9219 non-null   object 
 10  crew               9219 non-null   object 
 11  keywords           9219 non-null   object 
 12  crew_size          9219 non-null   int64  
 13  cast_size          9219 non-null   int64  
 14  director           9195 non-null   object 
dtypes: float64(2), int32(1), int64(2), object(10)
memory usage: 1.1+ MB


In [105]:
smol_data['cast']=smol_data['cast'].apply(lambda x:[i['name'] for i in x] if isinstance(x,list) else [])
smol_data['cast']=smol_data['cast'].apply(lambda x: x[:3] if len(x)>3 else x)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [106]:
smol_data['cast']=smol_data['cast'].apply(lambda x: x[:3] if len(x)>3 else x)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [107]:
smol_data

,genres,id,original_language,overview,popularity,tagline,title,vote_average,vote_count,cast,crew,keywords,crew_size,cast_size,director
0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,NaN,Toy Story,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",106,13,John Lasseter
1,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.0155,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",16,26,Joe Johnston
2,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"[Walter Matthau, Jack Lemmon, Ann-Margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o...",4,7,Howard Deutch
3,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.85949,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"[Whitney Houston, Angela Bassett, Loretta Devine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",10,10,Forest Whitaker
4,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.38752,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,"[Steve Martin, Diane Keaton, Martin Short]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug...",7,12,Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40952,[Drama],159550,en,A man must cope with the loss of his wife and ...,0.038998,NaN,The Last Brickmaker in America,7.0,1.0,"[Sidney Poitier, Wendy Crewson, Jay O. Sanders]","[{'credit_id': '544475aac3a36819fb000578', 'de...","[friendship, brick making]",2,7,Gregg Champion
41172,"[Thriller, Romance]",392572,hi,"Rustom Pavri, an honourable officer of the Ind...",7.333139,Decorated Officer. Devoted Family Man. Defendi...,Rustom,7.3,25.0,"[Akshay Kumar, Ileana D'Cruz, Esha Gupta]","[{'credit_id': '5951baf692514129c4016600', 'de...",[bollywood],16,14,Tinu Suresh Desai
41225,"[Adventure, Drama, History, Romance]",402672,hi,"Village lad Sarman is drawn to big, bad Mohenj...",1.423358,NaN,Mohenjo Daro,6.7,26.0,"[Hrithik Roshan, Pooja Hegde, Kabir Bedi]","[{'credit_id': '57cd5d3592514179d50018e8', 'de...",[bollywood],16,12,Ashutosh Gowariker
41391,"[Action, Adventure, Drama, Horror, Science Fic...",315011,ja,From the mind behind Evangelion comes a hit la...,9.285519,A god incarnate. A city doomed.,Shin Godzilla,6.6,152.0,"[Hiroki Hasegawa, Yutaka Takenouchi, Satomi Is...","[{'credit_id': '560892fa92514177550018b2', 'de...","[monster, godzilla, giant monster, destruction...",27,49,Hideaki Anno


In [108]:
key=smol_data.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1,drop=True)
key.name='keyword'

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [109]:
smol_data['cast']=smol_data['cast'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [110]:
smol_data['director']=smol_data['director'].astype(str).apply(lambda x: str.lower(x.replace(' ','')))
smol_data['director']=smol_data['director'].apply(lambda x: [x,x,x])

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [111]:
key=key.value_counts()

In [112]:
key[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [113]:
def filter_key(x):
    words=[]
    for i in x:
        if i in key:
            words.append(i)
    return words

In [114]:
smol_data['keywords']=smol_data['keywords'].apply(filter_key)

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [115]:
stemmer=SnowballStemmer('english')
smol_data['keywords']=smol_data['keywords'].apply(lambda x:[stemmer.stem(i) for i in x])
smol_data['keywords']=smol_data['keywords'].apply(lambda x:[str.lower(i.replace(' ','')) for i in x])

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [116]:
smol_data['dump']=smol_data['cast']+smol_data['director']+smol_data['keywords']+smol_data['genres']
smol_data['dump']=smol_data['dump'].apply(lambda x: ' '.join(x))

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [117]:
smol_data

,genres,id,original_language,overview,popularity,tagline,title,vote_average,vote_count,cast,crew,keywords,crew_size,cast_size,director,dump
0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.9469,NaN,Toy Story,7.7,5415.0,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousi, toy, boy, friendship, friend, rival...",106,13,"[johnlasseter, johnlasseter, johnlasseter]",tomhanks timallen donrickles johnlasseter john...
1,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.0155,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgam, disappear, basedonchildren'sbook, n...",16,26,"[joejohnston, joejohnston, joejohnston]",robinwilliams jonathanhyde kirstendunst joejoh...
2,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fish, bestfriend, duringcreditssting, oldmen]",4,7,"[howarddeutch, howarddeutch, howarddeutch]",waltermatthau jacklemmon ann-margret howarddeu...
3,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.85949,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[basedonnovel, interracialrelationship, single...",10,10,"[forestwhitaker, forestwhitaker, forestwhitaker]",whitneyhouston angelabassett lorettadevine for...
4,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.38752,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[babi, midlifecrisi, confid, age, daughter, mo...",7,12,"[charlesshyer, charlesshyer, charlesshyer]",stevemartin dianekeaton martinshort charlesshy...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40952,[Drama],159550,en,A man must cope with the loss of his wife and ...,0.038998,NaN,The Last Brickmaker in America,7.0,1.0,"[sidneypoitier, wendycrewson, jayo.sanders]","[{'credit_id': '544475aac3a36819fb000578', 'de...","[friendship, brickmak]",2,7,"[greggchampion, greggchampion, greggchampion]",sidneypoitier wendycrewson jayo.sanders greggc...
41172,"[Thriller, Romance]",392572,hi,"Rustom Pavri, an honourable officer of the Ind...",7.333139,Decorated Officer. Devoted Family Man. Defendi...,Rustom,7.3,25.0,"[akshaykumar, ileanad'cruz, eshagupta]","[{'credit_id': '5951baf692514129c4016600', 'de...",[bollywood],16,14,"[tinusureshdesai, tinusureshdesai, tinusureshd...",akshaykumar ileanad'cruz eshagupta tinusureshd...
41225,"[Adventure, Drama, History, Romance]",402672,hi,"Village lad Sarman is drawn to big, bad Mohenj...",1.423358,NaN,Mohenjo Daro,6.7,26.0,"[hrithikroshan, poojahegde, kabirbedi]","[{'credit_id': '57cd5d3592514179d50018e8', 'de...",[bollywood],16,12,"[ashutoshgowariker, ashutoshgowariker, ashutos...",hrithikroshan poojahegde kabirbedi ashutoshgow...
41391,"[Action, Adventure, Drama, Horror, Science Fic...",315011,ja,From the mind behind Evangelion comes a hit la...,9.285519,A god incarnate. A city doomed.,Shin Godzilla,6.6,152.0,"[hirokihasegawa, yutakatakenouchi, satomiishih...","[{'credit_id': '560892fa92514177550018b2', 'de...","[monster, godzilla, giantmonst, destruct, kaij...",27,49,"[hideakianno, hideakianno, hideakianno]",hirokihasegawa yutakatakenouchi satomiishihara...


In [118]:
cvt=CountVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
count_matrix=cvt.fit_transform(smol_data['dump'])

In [119]:
cosine_mat=cosine_similarity(count_matrix,count_matrix)

In [120]:
smol_data=smol_data.reset_index()
titles=smol_data['title']
indices=pd.Series(smol_data.index,index=smol_data['title'])

In [121]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
The Last Brickmaker in America                        9214
Rustom                                                9215
Mohenjo Daro                                          9216
Shin Godzilla                                         9217
The Beatles: Eight Days a Week - The Touring Years    9218
Length: 9219, dtype: int64

In [122]:
get_sugg('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
4145                      Insomnia
7648                     Inception
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

In [123]:
def improved_rec(title):
    idx=indices[title]
    sim_scores=list(enumerate(cosine_mat[idx]))
    sim_scores=sorted(sim_scores,key=lambda x:x[1],reverse=True)
    sim_scores=sim_scores[1:31]
    index=[i[0] for i in sim_scores]
    
    movies=smol_data.iloc[index][['title','vote_average','vote_count']]
    m=movies['vote_count'].quantile(0.6)
    C=movies['vote_average'].mean()
    qualified=movies[(movies['vote_count']>=m)&(movies['vote_count'].notnull())&(movies['vote_average'].notnull())]
    qualified['vote_count']=qualified['vote_count'].astype(int)
    qualified['vote_average']=qualified['vote_average'].astype(int)
    qualified['wrd']=qualified.apply(Weightedrating,axis=1)
    qualified=qualified.sort_values('wrd',ascending=False).head(10)
    return qualified

In [124]:
improved_rec('The Dark Knight')

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,title,vote_average,vote_count,wrd
7648,Inception,8,14075,7.928755
8613,Interstellar,8,11187,7.911049
6623,The Prestige,8,4510,7.790919
3381,Memento,8,4168,7.775381
8031,The Dark Knight Rises,7,9263,6.938156
6218,Batman Begins,7,7511,6.924519
1134,Batman Returns,6,1706,5.922571
4145,Insomnia,6,1181,5.897401
2131,Superman,6,1042,5.887738
132,Batman Forever,5,1529,5.136680


In [128]:
improved_rec('Midnight Express')

C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Haikrishnan\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,title,vote_average,vote_count,wrd
284,The Shawshank Redemption,8,8358,7.882427
6881,American Gangster,7,1527,6.694188
4550,The Life of David Gale,7,497,6.355856
1068,Pink Floyd: The Wall,7,363,6.247556
3451,Mississippi Burning,7,336,6.221171
2976,Angel Heart,7,326,6.210924
1044,Down by Law,7,149,5.971358
8890,True Story,6,536,5.829177
1346,The Rainmaker,6,239,5.753792
252,Murder in the First,6,140,5.711327
